# TF-IDF notebook

In this notebook we will build a first method to find the frequency of words in a corpus and see their relevance.

We will work with the corpus of EHR (CSV files - Lounes), and then build the TF and then IDF model.

WE will then try to have visualization aspect.

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Download the csv file to analyze
TEST = {1: "le tf-idf de l'anglais term frequency-inverse document frequency est une méthode de pondération souvent utilisée en recherche d'information et en particulier dans la fouille de textes",
       2: "cette mesure statistique permet d'évaluer l'importance d'un terme contenu dans un document relativement à une collection ou un corpus",
       3: "le poids augmente proportionnellement au nombre d'occurrences du mot dans le document",
       4: "il varie également en fonction de la fréquence du mot dans le corpus ainsi le",
       5: "des variantes de la formule originale sont souvent utilisées dans des moteurs de recherche pour apprécier la pertinence d'un document en fonction des critères de recherche de l'utilisateur"}
df = pd.DataFrame.from_dict(TEST, orient = 'index')
#df[1] = [element[0].split() for _, element in df.iterrows()]
all_words = set([word for _, words in df.iterrows() for word in words[0].split() for _, words in df.iterrows()])
all_words_dict = {word:0 for word in all_words}
#df

## Bag of words

In [24]:
def raw_count_text(doc):
    """
    Given a text, return a dictionnary with the amount of occurences of a word
    """
    words = {}
    for w in doc.split():
        if w not in words:
            words[w] = 1
        else:
            words[w]+=1
    return words

dict_df = {}
i=0
for k, sentence in TEST.items():
    dict_df[i] = raw_count_text(sentence)
    i+=1

df  = pd.DataFrame.from_dict(dict_df, orient = 'index')
df


,le,tf-idf,de,l'anglais,term,frequency-inverse,document,frequency,est,une,...,formule,originale,sont,utilisées,moteurs,pour,apprécier,pertinence,critères,l'utilisateur
0,1.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,4.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## TF aspect

In [3]:
# Finding the count in each document
def raw_count_text(doc):
    """
    Given a text, return a dictionnary with the amount of occurences of a word
    """
    words = {}
    for w in doc.split():
        if w not in words:
            words[w] = 1
        else:
            words[w]+=1
    return words
    
def term_frequency_text(doc, log = False):
    """
    Given a text, return a dictionnary with the frequency of a word within the text
    """
    words = {}
    for w in doc.split():
        if w not in words:
            words[w] = 1
        else:
            words[w] +=1
    if log == True:
        words = {k:np.log(1+v) for k,v in words.items()}
    else:
        words = {k:(v*100)/sum(words.values()) for k,v in words.items()}
    return words

def bool_frequency(doc):
    words = {}
    for w in all_words:
        if w in doc.split():
            words[w] = 1
        else:
            words[w] = 0
    return words

# Finding the count for the whole corpus
def raw_count_corpus(data):
    """
    Given a data frame composed of text, return a dictionnary with the amount of occurences of a word
    """
    words = {}
    for _, string in data.iterrows():
        for w in string[0].split():
            if w not in words:
                words[w] = 1
            else:
                words[w]+=1
    return words
    
def term_frequency_corpus(data, log = False):
    """
    Given a dataframe composed of texts, return a dictionnary with the frequency of a word within the corpus
    """
    words = {}
    for _, string in data.iterrows():
        for w in string[0].split():
            if w not in words:
                words[w] = 1
            else:
                words[w] +=1
    if log == True:
        words = {k:np.log(1+v) for k,v in words.items()}
    else:
        words = {k:(v*100)/sum(words.values()) for k,v in words.items()}
    return words

## IDF aspect

In [18]:
# Compute the IDF
def idf(data):
    N = data.shape[0]
    frequency = {}
    for _, word in data.iterrows():
        bf = bool_frequency(word[0])
        for k,v in bf.items():
            if v == 1:
                if k not in frequency:
                    frequency[k] = 1
                else:
                    frequency[k] +=1
    return frequency


## TF-IDF together

In [ ]:
tf_idf = {}
idf_data = idf(df)
tfidf = {i:{word:0 for word, _ in idf_data.items()} for i in range(df.shape[0])}
i=0
for _, text in df.iterrows():
    a = term_frequency_text(text[0])
    for word, freq in a.items():
        tfidf[i][word] = freq * idf_data[word]
    i+=1
pd.DataFrame.from_dict(tfidf, orient = 'index')

,fouille,frequency,dans,utilisée,tf-idf,est,souvent,textes,document,d'information,...,pour,des,originale,moteurs,l'utilisateur,utilisées,pertinence,variantes,sont,critères
0,3.846154,3.846154,19.230769,3.846154,3.846154,3.846154,7.692308,3.846154,15.384615,3.846154,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,26.315789,0.000000,0.000000,0.000000,0.000000,0.000000,21.052632,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,41.666667,0.000000,0.000000,0.000000,0.000000,0.000000,33.333333,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,33.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,17.857143,0.000000,0.000000,0.000000,7.142857,0.000000,14.285714,0.000000,...,3.571429,10.714286,3.571429,3.571429,3.571429,3.571429,3.571429,3.571429,3.571429,3.571429


## TF-IDF Vectorizer

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [7]:
corpus = list(TEST.values())

In [8]:
corpus

["le tf-idf de l'anglais term frequency-inverse document frequency est une méthode de pondération souvent utilisée en recherche d'information et en particulier dans la fouille de textes",
 "cette mesure statistique permet d'évaluer l'importance d'un terme contenu dans un document relativement à une collection ou un corpus",
 "le poids augmente proportionnellement au nombre d'occurrences du mot dans le document",
 'il varie également en fonction de la fréquence du mot dans le corpus ainsi le',
 "des variantes de la formule originale sont souvent utilisées dans des moteurs de recherche pour apprécier la pertinence d'un document en fonction des critères de recherche de l'utilisateur"]

In [9]:
vectorizer = TfidfVectorizer()

In [10]:
X = vectorizer.fit_transform(corpus)

In [11]:
vectorizer.get_feature_names_out()

array(['ainsi', 'anglais', 'apprécier', 'au', 'augmente', 'cette',
       'collection', 'contenu', 'corpus', 'critères', 'dans', 'de', 'des',
       'document', 'du', 'en', 'est', 'et', 'fonction', 'formule',
       'fouille', 'frequency', 'fréquence', 'idf', 'il', 'importance',
       'information', 'inverse', 'la', 'le', 'mesure', 'mot', 'moteurs',
       'méthode', 'nombre', 'occurrences', 'originale', 'ou',
       'particulier', 'permet', 'pertinence', 'poids', 'pondération',
       'pour', 'proportionnellement', 'recherche', 'relativement', 'sont',
       'souvent', 'statistique', 'term', 'terme', 'textes', 'tf', 'un',
       'une', 'utilisateur', 'utilisée', 'utilisées', 'variantes',
       'varie', 'également', 'évaluer'], dtype=object)

In [12]:
print(X.shape)

(5, 63)


In [13]:
X

<5x63 sparse matrix of type '<class 'numpy.float64'>'
	with 86 stored elements in Compressed Sparse Row format>